In [18]:
# Importacion de librerias para manipulacion de datos
import pandas as pd
import numpy as np

In [19]:
# Importacion de Plotly como libreria de DataViz
import plotly.graph_objects as go

In [30]:
# Importacion de Sklearn para la costruccion de modelo
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve, train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

#### Configurando path

In [21]:
import os

# Obtiene la ubicación completa del archivo del notebook
notebook_path = os.path.realpath('__file__')

# Obtiene la ruta absoluta de la carpeta "Coderhouser-DS-TP0002" sin importar donde te encuentres
path_base = os.path.abspath(os.path.join(notebook_path, "..", "..", "..", "Coderhouser-DS-TP0002"))

print("Path base':", path_base)

Path base': C:\develoment\Coderhouser-DS-TP0002


#### Leyendo datos:

In [47]:
# Generando el Dataframe apartir de los dataset RedTeam y BlueTeam
blueTeam = pd.read_csv( path_base + r"\\data\procesed\blueTeam.csv")
blueTeam["Team"] = "Blue"
redTeam = pd.read_csv( path_base + r"\\data\procesed\redTeam.csv")
redTeam["Team"] = "Red"
data = pd.concat([blueTeam,redTeam])

In [48]:
# Reemplazar los valores de la columna "Wins"
data['WinTags'] = data['Wins'].replace({0: 'Loss', 1: 'Win'})

# Crear un mapeo de colores
color_map = {'Win': 'yellow', 'Loss': 'gray'}
data.drop(columns=["Unnamed: 0"], inplace=True)

### Encoding:

In [49]:
data = pd.get_dummies(data, columns=['Team'])

### Feature Engineering:

1. Creación de Características Derivadas:

In [89]:
data['KillRate'] = data['Kills'] / data['AvgLevel']
data['NormalizedGoldExperienceDiff'] = (data['GoldDiff'] - data['ExperienceDiff']) / data['AvgLevel']

2. Manejo de Variables Temporales:

3. Interacción entre Variables:

In [43]:
data['GoldPerKill'] = data['TotalGold'] / data['Kills']

4. Análisis de Correlación:

In [27]:
correlation_matrix = data.corr()

C:\Users\Fernando\AppData\Local\Temp\ipykernel_14656\2558734126.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = data.corr()


5. Normalización de Variables:

In [28]:

scaler = MinMaxScaler()
data[['TotalGold', 'AvgLevel']] = scaler.fit_transform(data[['TotalGold', 'AvgLevel']])

### Separación de Datos:
- Divide tu conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba para evaluar el rendimiento de tu modelo. Puedes usar, por ejemplo, un 80% de los datos para entrenamiento y un 20% para prueba.

In [50]:
X = data.drop(columns=['WinTags'])  # X serán las características, excluyendo la columna objetivo (Wins)
y = data['WinTags']  # y será la variable objetivo (Wins)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Entrenamiento de un Modelo de Machine Learning:


In [82]:
# Define tu modelo base
base_model = RandomForestClassifier(random_state=42)
base_model.fit(X_train, y_train)
y_pred_base = base_model.predict(X_test)
accuracy_base = accuracy_score(y_test, y_pred_base)

In [91]:
feature_combinations = [
    ['Kills', 'Assists', 'TotalGold', 'GoldDiff', 'CSPerMin', 'GoldPerMin','WinTags'],
    ['FirstBlood', 'Kills', 'Deaths', 'Assists','WinTags'],
    ['TowersDestroyed', 'TotalGold', 'TotalExperience', 'GoldDiff', 'ExperienceDiff', 'CSPerMin', 'GoldPerMin','WinTags']
]
accuracies = [0.75, 0.75, 0.75 ]


In [90]:
list(data.columns.values)

['Wins',
 'WardsPlaced',
 'WardsDestroyed',
 'FirstBlood',
 'Kills',
 'Deaths',
 'Assists',
 'EliteMonsters',
 'Dragons',
 'Heralds',
 'TowersDestroyed',
 'TotalGold',
 'AvgLevel',
 'TotalExperience',
 'TotalMinionsKilled',
 'TotalJungleMinionsKilled',
 'GoldDiff',
 'ExperienceDiff',
 'CSPerMin',
 'GoldPerMin',
 'WinTags',
 'Team_Blue',
 'Team_Red',
 'KillRate',
 'NormalizedGoldExperienceDiff']

In [92]:

# Itera sobre combinaciones de ingeniería de características
for feature_combination in feature_combinations:
    # Aplica la ingeniería de características a X_train y X_test según feature_combination
    x_data = data[feature_combination]
    X = x_data.drop(columns=['WinTags'])  # X serán las características, excluyendo la columna objetivo (Wins)
    y = x_data['WinTags']  # y será la variable objetivo (Wins)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrena el modelo con las características transformadas
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Realiza predicciones y evalúa el rendimiento
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Almacena los resultados
    feature_combinations.append(str(feature_combination))
    accuracies.append(accuracy)
    print(accuracy)

0.6973684210526315
0.6632085020242915
0.7034412955465587


KeyError: "['Kills', 'Assists', 'TotalGold', 'GoldDiff', 'CSPerMin', 'GoldPerMin', 'WinTags']"

### Evaluación del Modelo:

In [93]:
# Crea la figura interactiva de Plotly
fig = go.Figure()

# Agrega la barra del modelo base
fig.add_trace(go.Bar(x=['Base Model'], y=[accuracy_base], name='Base Model'))

# Agrega barras para cada combinación de ingeniería de características
fig.add_trace(go.Bar(x=feature_combinations, y=accuracies, name='Feature Engineering Combinations'))

# Diseño del gráfico
fig.update_layout(
    title='Mejora del Rendimiento con Ingeniería de Características',
    xaxis_title='Combinación de Ingeniería de Características',
    yaxis_title='Precisión',
    barmode='group',
    legend=dict(x=0.02, y=0.95),
    margin=dict(l=0, r=0, t=30, b=0)
)

# Muestra la figura interactiva
fig.show()


### Ajuste y Optimización del Modelo:
- Si el rendimiento no es satisfactorio, puedes ajustar hiperparámetros, probar diferentes algoritmos o realizar ingeniería de características adicional para mejorar el modelo.

In [ ]:
"""
# Obtén las probabilidades de predicción para la clase positiva (Win)
y_probs = best_model.predict_proba(X_test)[:, 1]

# Calcula la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

# Crea la figura interactiva de Plotly
fig = go.Figure()

# Agrega la curva ROC
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='Curva ROC (AUC = {:.2f})'.format(roc_auc)))

# Línea de referencia
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Línea de Referencia', line=dict(dash='dash')))

# Diseño del gráfico
fig.update_layout(
    title='Curva ROC Random Forest',
    xaxis_title='Tasa de Falsos Positivos (FPR)',
    yaxis_title='Tasa de Verdaderos Positivos (TPR)',
    legend=dict(x=0.02, y=0.95),
    margin=dict(l=0, r=0, t=30, b=0)
)

# Muestra la figura interactiva
fig.show()
"""

### Conclusión:
A pesar de que tanto el modelo base como el mejor modelo lograron una eficacia del 100%, estamos actualmente en la fase de evaluación de usabilidad para detectar posibles casos de sobreajuste. Se observa claramente una señal de sobreajuste debido a la presencia de registros anómalos, comúnmente conocidos como "Remontadas épicas". A pesar de la consistencia del equipo perdedor durante los primeros 30 minutos, hay instancias donde el equipo, inicialmente en desventaja, muestra una notable persistencia, revirtiendo el curso de la partida y siendo catalogado por el modelo como perdedor. Este fenómeno desafía la lógica del modelo al reconocer la victoria del equipo que, según su evaluación, debería haber perdido.

### Recursos
- [Use Random Forest to predict on Walmart Sales](https://medium.com/mlearning-ai/use-random-forest-to-predict-on-walmart-sales-ae6ebadb569b)
- [Isolation Forest](https://medium.com/@corymaklin/isolation-forest-799fceacdda4)
- [ML Dataset — How To Prepare a Good One?](https://betterprogramming.pub/ml-dataset-how-to-prepare-a-good-one-7d92ce1d45e5)
- [Use Random Forest to predict on a multiclass imbalance problem](https://medium.com/mlearning-ai/use-random-forest-to-predict-on-a-multiclass-imbalance-problem-fd0259098ac4)